# install stuff

In [1]:
# https://getfile.dokpub.com/yandex/
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/iwvBZBEbSxrxKA
!mv iwvBZBEbSxrxKA hw_data.zip
!unzip -qq \*.zip
!ls

connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s73vlx.storage.yandex.net/rdisk/c210e36198b1dca97ce5536888cea5bb54ad1ff14eccb49e0b33f2f991dd4545/648601e8/zRSoMcrzFZAW_dPFYqsjx_KBInBOWlW_Y6J58u9RXwEdaVyzd_SdFUVi2KBA63vuv2olNLMmZ5pLuSl_RqcnUg==?uid=0&filename=hw_data.zip&disposition=attachment&hash=SxFOJWP5EtYiwMY6efCvft3ej0uItcqyAJOZBQe/NxqNYZ8EChgGhZ/8eiE2OsRdq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=225893658&fsize=763284441&hid=0250d3dd2aec14e3a9b72b4afa018e11&media_type=compressed&tknv=v2&rtoken=pVmgNHLCOWHq&force_default=no&ycrid=na-98f450600d1d48deb9eeefc90f6ca7c3-downloader1f&ts=5fdddca964a00&s=55093e257a908ccb66f94c4b533f629d10a03339bc82bdebcfb83af86dc206ef&pb=U2FsdGVkX19c5Ipcqd8UbpDYQdnc6AHP96LpwFHXCF5VP79d788v3rMaonAqPdk7qKlL5g_bwC8N9OjCvS5ufD9XzNwGt6lT3CsHnB6sKKE [following]
--2023-06-11 14:45:15--  https://s73vlx.storage.yandex.net/rdisk/c210e36198b1dca97ce5536888cea5bb54ad1ff14eccb49e0b33f2f991dd4545/6486

In [2]:
!pip3 install catboost
!pip3 install emoji==2.4.0
!pip3 install pyspark pandas scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=31dc141c91dee92f60f73671d64bbf09119a70b2afeb3234b122528d9926fe45
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256

# setup pyspark

In [3]:
import pyspark.sql
from pyspark.sql import functions as sf
import pyspark

from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number, lag
from pyspark.sql.functions import to_date, to_timestamp, datediff
from pyspark.sql.functions import udf

from pyspark.sql.types import IntegerType, BooleanType

In [4]:
import emoji

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

In [5]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "512m") \
    .set("spark.driver.memory", "512m")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

# load data

In [6]:
!ls

channels_orc  hw_data.zip  posts_test.orc  posts_train.orc  sample_data


In [7]:
data_path = './hw_data/'
data_path = './'

train_data = ss.read.orc(f'{data_path}posts_train.orc')
test_data = ss.read.orc(f'{data_path}posts_test.orc')
channel_data = ss.read.orc(f'{data_path}channels_orc')

# explore data

In [8]:
train_data.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:...|        null|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:...|        null|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:...|        null|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:...|        null|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:...|        null|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------

In [9]:
channel_data.limit(5).show()

+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|         description|is_private|         last_parsed|            name|post_count|     tg_id|               title|             updated|user_count|channel_id|
+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|                    |     false|2019-01-26 16:53:...|   MoeinZchannel|     708.0|1002972402|             Moein Z|2019-01-26 16:53:...|   62411.0|      7910|
|Вокруг столько ме...|     false|2019-02-12 00:39:...|  merzotachannel|    1027.0|1336284461|            Мерзость|2019-02-12 00:39:...|   12982.0|     14121|
|🗣እኛስ የተሰቀለውን ክርስ...|     false|2019-02-10 06:57:...|christian_mezmur|    1168.0|1136987361|Christian Mezmur ...|2019-02-10 06:57:...|   21704.0|     17375|
|     Chiroyli_qomatt|     false|2018-12-08 00:54:...

In [10]:
train_data = train_data.withColumn('date', to_timestamp(train_data.date))
test_data = test_data.withColumn('date', to_timestamp(test_data.date))

In [11]:
assert train_data.join(channel_data, 'channel_id').count() == train_data.count()
assert test_data.join(channel_data, 'channel_id').count() == test_data.count()

# make features

The final list of features is as follows:

| № | Description                              | Category                      |
|---|------------------------------------------|-------------------------------|
| 1 | Sequential number of a post in a channel | Window function: `row_number` |
| 2 | Days since the previous post             | Window function: `lag`        |
| 3 | Number of subscribers in channel         | Uses channels' metadata       |
| 4 | Number of emojis in a post               | Uses text field               |

In [12]:
train_data_sample = train_data.limit(1000)

## Feature 1
Sequential number of a post in a channel

In [13]:
channel_window  = Window.partitionBy("channel_id").orderBy("date")

# example of the feature on a subsample of data
train_data_sample.withColumn("post_num", row_number().over(channel_window)) \
.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+--------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|               date|forwarded_id|post_num|
+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+--------+
|       221|40129279|  388|Роль нашего скром...| 2096|        f|           f|2016-11-15 15:29:45|        null|       1|
|       221|40129165|  558|Дима, а у меня во...| 2103|        f|           f|2016-12-31 08:50:19|        null|       2|
|       221|  583250|  603|В докладе, опубли...| 3241|        f|           f|2017-01-11 19:25:58|        null|       3|
|       221|  583170|  637|Про Методичку и ч...| 3590|        f|           f|2017-01-14 14:44:14|        null|       4|
|       221|  583088|  656|К вопросу о ювена...| 2585|        f|           f|2017-01-16 13:53:25|        null|       5|
+----------+--------+-----+-------------

## Feature 2

Days since the previous post

In [14]:
channel_window  = Window.partitionBy("channel_id").orderBy("date")

# example of the feature on a subsample of data
train_data_sample.withColumn("previous_post_date", lag("date", 1).over(channel_window)) \
          .withColumn('days_since_prev', datediff(col('date'), col('previous_post_date'))) \
          .fillna(value=0, subset=['days_since_prev']) \
.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+-------------------+---------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|               date|forwarded_id| previous_post_date|days_since_prev|
+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+-------------------+---------------+
|       221|40129279|  388|Роль нашего скром...| 2096|        f|           f|2016-11-15 15:29:45|        null|               null|              0|
|       221|40129165|  558|Дима, а у меня во...| 2103|        f|           f|2016-12-31 08:50:19|        null|2016-11-15 15:29:45|             46|
|       221|  583250|  603|В докладе, опубли...| 3241|        f|           f|2017-01-11 19:25:58|        null|2016-12-31 08:50:19|             11|
|       221|  583170|  637|Про Методичку и ч...| 3590|        f|           f|2017-01-14 14:44:14|        null|2017-01-

## Feature 3

Number of subscribers in channel

In [15]:
# example of the feature on a subsample of data
train_data_sample.join(channel_data, 'channel_id').select(train_data["*"], channel_data["user_count"]) \
.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+----------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|               date|forwarded_id|user_count|
+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+----------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:21|        null|    8732.0|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:00|        null|    8732.0|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:52|        null|    8732.0|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:38|        null|    8732.0|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:56|        null|    8732.0|
+----------+--------+---

## Feature 4

Number of emojis in a post

In [16]:
emoji_counter = udf(lambda x: emoji.emoji_count(x), IntegerType())

# example of the feature on a subsample of data
train_data_sample.fillna(value='', subset=['text']) \
.withColumn("emojis", emoji_counter(col("text"))) \
.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|               date|forwarded_id|emojis|
+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:21|        null|     0|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:00|        null|     0|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:52|        null|     0|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:38|        null|     0|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:56|        null|     0|
+----------+--------+-----+--------------------+-----+--

## Additional features

These features should not be graded and are only used for views prediction.

In [17]:
strlen = udf(lambda x: len(x), IntegerType())

# example of the feature on a subsample of data
train_data_sample.withColumn("post_length", strlen(col("text"))) \
.fillna(value=0, subset=['post_length']) \
.limit(5).show()

+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+-----------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|               date|forwarded_id|post_length|
+----------+--------+-----+--------------------+-----+---------+------------+-------------------+------------+-----------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:21|        null|         60|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:00|        null|        193|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:52|        null|        207|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:38|        null|        144|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:56|        null|        230|
+----------+----

## Feature generation

Let's wrap all new features into `make_features` helper function:

In [18]:
def make_feature_1(df):
  channel_window  = Window.partitionBy("channel_id").orderBy("date")
  return df.withColumn("post_num", row_number().over(channel_window))

def make_feature_2(df):
  channel_window  = Window.partitionBy("channel_id").orderBy("date")

  return df.withColumn("previous_post_date", lag("date", 1).over(channel_window)) \
            .withColumn('days_since_prev', datediff(col('date'), col('previous_post_date'))) \
            .fillna(value=0, subset=['days_since_prev']) \

def make_feature_3(df):
  return df.join(channel_data, 'channel_id').select(df["*"], channel_data["user_count"])

def make_feature_4(df):
  emoji_counter = udf(lambda x: emoji.emoji_count(x), IntegerType())

  return df.fillna(value='', subset=['text']) \
  .withColumn("emojis", emoji_counter(col("text")))

def make_additional_features(df):
  strlen = udf(lambda x: len(x), IntegerType())
  
  return df.withColumn("post_length", strlen(col("text"))) \
  .fillna(value=0, subset=['post_length'])

def make_features(df):

    cols = ['has_image', 'is_forwarded', 'post_num', 'days_since_prev', 'user_count', 'emojis', 'post_length']
    cols += ['id']

    out_df = make_feature_1(df)
    out_df = make_feature_2(out_df)
    out_df = make_feature_3(out_df)
    out_df = make_feature_4(out_df)
    out_df = make_additional_features(out_df)

    if 'views' in out_df.columns:
        cols.append('views')
    return out_df.select(*cols)

In [19]:
train_features = make_features(train_data).cache()
test_features = make_features(test_data).cache()

In [20]:
train_features.write.csv('train_csv', mode='overwrite', header=True)
test_features.write.csv('test_csv', mode='overwrite', header=True)

In [21]:
train_features.limit(10).show()

+---------+------------+--------+---------------+----------+------+-----------+--------+-----+
|has_image|is_forwarded|post_num|days_since_prev|user_count|emojis|post_length|      id|views|
+---------+------------+--------+---------------+----------+------+-----------+--------+-----+
|        t|           f|       1|              0|    4868.0|     0|          0|39957267| 1217|
|        t|           f|       2|              1|    4868.0|     0|          0|39957266| 1143|
|        t|           f|       3|              0|    4868.0|     0|          0|39957265| 1127|
|        t|           f|       4|              1|    4868.0|     0|          0|39957264| 1135|
|        f|           f|       5|              0|    4868.0|     0|          0|39957263| 1286|
|        t|           f|       6|              1|    4868.0|     0|          0|39957262| 1264|
|        t|           f|       7|              2|    4868.0|     0|          0|39957261| 1230|
|        f|           f|       8|              0| 

# load features to pandas
you also can use .toPandas()

In [22]:
import subprocess
import glob
import os
import shutil
import pandas as pd

# load_and_merge_csv
def load_and_merge_csv_from_hdfs(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')

    res['has_image'] = res['has_image'].apply(lambda x: x=='t')
    res['is_forwarded'] = res['is_forwarded'].apply(lambda x: x=='t')
    return res

In [23]:
trainXY = load_and_merge_csv_from_hdfs('train_csv')
testX = load_and_merge_csv_from_hdfs('test_csv')

In [24]:
import numpy as np

Ycol = 'views'
to_drop = []
trainX, trainY = trainXY.drop(Ycol, axis=1).drop(to_drop, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)

testX = testX.drop(to_drop, axis=1)

# train your model and predict test

I will use catboost to predict the number of views in a post.

First, let's check model's performance using train-test split.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.1, random_state=12)

model = CatBoostRegressor(iterations=20,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_test)

0:	learn: 1.3326665	total: 548ms	remaining: 10.4s
1:	learn: 1.3094981	total: 927ms	remaining: 8.34s
2:	learn: 1.2926029	total: 1.42s	remaining: 8.03s
3:	learn: 1.2849776	total: 2.03s	remaining: 8.13s
4:	learn: 1.2783807	total: 2.7s	remaining: 8.1s
5:	learn: 1.2750172	total: 3.38s	remaining: 7.89s
6:	learn: 1.2721577	total: 4.05s	remaining: 7.53s
7:	learn: 1.2700326	total: 4.74s	remaining: 7.11s
8:	learn: 1.2687277	total: 5.39s	remaining: 6.59s
9:	learn: 1.2678183	total: 6.04s	remaining: 6.04s
10:	learn: 1.2660261	total: 6.73s	remaining: 5.51s
11:	learn: 1.2629707	total: 7.38s	remaining: 4.92s
12:	learn: 1.2624346	total: 8.01s	remaining: 4.31s
13:	learn: 1.2614369	total: 8.39s	remaining: 3.6s
14:	learn: 1.2605656	total: 8.74s	remaining: 2.91s
15:	learn: 1.2599384	total: 9.12s	remaining: 2.28s
16:	learn: 1.2594745	total: 9.49s	remaining: 1.68s
17:	learn: 1.2583758	total: 9.86s	remaining: 1.09s
18:	learn: 1.2574079	total: 10.2s	remaining: 539ms
19:	learn: 1.2564662	total: 10.6s	remaining:

In [26]:
def eval_errors(true, pred):
  errors = [
      mean_absolute_percentage_error(true, pred),
      mean_absolute_error(true, pred),
      mean_squared_error(true, pred),
      np.sqrt(mean_squared_error(true, pred))

  ]
  baseline = [15.707128974856676, 1.219070382113261, 2.4324378881170055, 1.5596274837655963]
  names = ['MAPE', 'MAE', 'MSE', 'RMSE']

  out = pd.DataFrame({'metric': names, 'baseline': baseline, 'value': errors})
  out['ok?'] = out['baseline'] > out['value']

  return out


eval_errors(y_test, preds)

,metric,baseline,value,ok?
0,MAPE,15.707129,0.125627,True
1,MAE,1.219070,0.931305,True
2,MSE,2.432438,1.581049,True
3,RMSE,1.559627,1.257398,True


It seems that the model is OK and it can be used for the submission.

In [27]:
model = CatBoostRegressor(iterations=20,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(trainX, trainY)
# Get predictions
prediction = model.predict(testX)

0:	learn: 1.3332240	total: 853ms	remaining: 16.2s
1:	learn: 1.3109340	total: 1.3s	remaining: 11.7s
2:	learn: 1.2940124	total: 1.66s	remaining: 9.43s
3:	learn: 1.2862716	total: 2.02s	remaining: 8.1s
4:	learn: 1.2791381	total: 2.43s	remaining: 7.28s
5:	learn: 1.2756759	total: 2.81s	remaining: 6.55s
6:	learn: 1.2728337	total: 3.22s	remaining: 5.97s
7:	learn: 1.2714706	total: 3.57s	remaining: 5.36s
8:	learn: 1.2703775	total: 3.95s	remaining: 4.82s
9:	learn: 1.2689006	total: 4.3s	remaining: 4.3s
10:	learn: 1.2677930	total: 4.66s	remaining: 3.81s
11:	learn: 1.2669651	total: 5.03s	remaining: 3.35s
12:	learn: 1.2659733	total: 5.43s	remaining: 2.92s
13:	learn: 1.2651549	total: 5.8s	remaining: 2.49s
14:	learn: 1.2637859	total: 6.17s	remaining: 2.06s
15:	learn: 1.2628738	total: 6.55s	remaining: 1.64s
16:	learn: 1.2622537	total: 6.89s	remaining: 1.22s
17:	learn: 1.2617612	total: 7.26s	remaining: 807ms
18:	learn: 1.2610219	total: 7.61s	remaining: 401ms
19:	learn: 1.2594568	total: 7.96s	remaining: 0

In [28]:
assert prediction.shape == (244386,)

# submit

- login: `user45`
- password: `10478742`

In [29]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2023/src/client.py -o client.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1907  100  1907    0     0   4314      0 --:--:-- --:--:-- --:--:--  4314


In [30]:
import client

In [32]:
# submit = pd.DataFrame({'views': prediction}, index=testX.index)
submit = pd.DataFrame({'views': prediction}, index=testX.index)

client.make_eval(submit, final=True)

{'data': {'mape': 15.126409621416842,
  'mean_absolute_error': 1.1660864585844264,
  'mean_squared_error': 2.335223079482057,
  'rmse': 1.5281436710866085,
  'rmspe': 23.146845918598878},
 'ok': True}

In [33]:
client.check_results()

{'2023-06-05': [{'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 15.077921763867085,
    'mean_absolute_error': 1.1509313693318939,
    'mean_squared_error': 2.3502023823870664,
    'rmse': 1.5330369801107429,
    'rmspe': 23.68953215186946}},
  {'baseline_beaten': True,
   'is_final': False,
   'metrics': {'mape': 14.991755949717588,
    'mean_absolute_error': 1.152580072547508,
    'mean_squared_error': 2.3072814461419027,
    'rmse': 1.5189738135142101,
    'rmspe': 23.1136310222326}}],
 '2023-06-11': [{'baseline_beaten': True,
   'is_final': False,
   'metrics': {'mape': 15.004556010027693,
    'mean_absolute_error': 1.1528927280576837,
    'mean_squared_error': 2.2956951708442572,
    'rmse': 1.515155163949969,
    'rmspe': 23.08565355313524}},
  {'baseline_beaten': True,
   'is_final': True,
   'metrics': {'mape': 15.004556010027693,
    'mean_absolute_error': 1.1528927280576837,
    'mean_squared_error': 2.2956951708442572,
    'rmse': 1.515155163949969,
 